
<div dir="rtl" align="right">

# 📘 اسلات A-1: پشت‌صحنهٔ FastAPI — مبانی HTTP و تست `/people`

**هدف جلسه (۴۵ دقیقه):**
- فهم عملی **HTTP Request/Response** در حد نیاز FastAPI.
- اجرای یک سرویس کوچک `/people` و تست آن با **curl** (نمایش Headers/Status/Body).
- جایگاه **FastAPI** (Application)، **Uvicorn** (ASGI Server) و **curl** (Client) در زنجیرهٔ درخواست–پاسخ.

**نقشهٔ مفهومی کوتاه:**
کلاینت (curl/مرورگر) → HTTP Request → سوکت شبکه → **Uvicorn** (ASGI Server) → **FastAPI** (Routing/Validation/Response) → HTTP Response → کلاینت

</div>



<div dir="rtl" align="right">

## ۱) مبانی HTTP — Request و Response

**Request** سه بخش اصلی دارد:
1. **Request Line** مثل `GET /people HTTP/1.1`
2. **Headers** مثل `Host`, `User-Agent`, `Accept`
3. **Body** (در متدهایی مانند `POST/PUT/PATCH`)

**Response** نیز سه بخش دارد:
1. **Status Line** مثل `HTTP/1.1 200 OK`
2. **Headers** مانند `Content-Type: application/json`
3. **Body** (مثلاً JSON)

**نکتهٔ کلیدی:** هدر `Content-Type` به کلاینت می‌گوید بدنه چه فرمتی دارد (JSON/HTML/…)، تا بتواند درست تفسیر کند.

</div>


In [ ]:
# main.py
from fastapi import FastAPI

app = FastAPI()


@app.get("/people")
def read_people():
    return {"people": ["Ali", "Sara", "Reza"]}


<div dir="rtl" align="right">

## ۲) اجرای سرور با Uvicorn (ASGI Server)

در ترمینال پروژه اجرا کنید:
```
uvicorn main:app --reload
```
- `main:app` یعنی از فایل `main.py`، آبجکت `app` را بارگذاری کن.
- `--reload` مخصوص توسعه است (با هر ذخیره، سرور ری‌لود می‌شود).
- سرور پیش‌فرض روی **پورت 8000** گوش می‌دهد.

پس از اجرا، در مرورگر تست کنید:
```
http://127.0.0.1:8000/people
```

</div>



<div dir="rtl" align="right">

## ۳) تست با `curl` و نکتهٔ PowerShell

در یک **ترمینال جدا** (کلاینت)، هدرها + بدنهٔ پاسخ را ببینید:
```
curl -i http://127.0.0.1:8000/people
```
خروجی نمونه:
```
HTTP/1.1 200 OK
date: Tue, 17 Sep 2025 10:00:00 GMT
server: uvicorn
content-type: application/json
content-length: 36

{"people":["Ali","Sara","Reza"]}
```

**PowerShell (ویندوز):** چون `curl` در PowerShell alias است، بهتر است بزنید:
```
curl.exe -i http://127.0.0.1:8000/people
```
یا معادل PowerShell:
```
Invoke-WebRequest -Uri http://127.0.0.1:8000/people
```
(این آخری معمولاً فقط Body را نشان می‌دهد.)

</div>



<div dir="rtl" align="right">

## ۴) موشکافی هدرها (Headers)

**Request Headers** نمونه:
- `Host: 127.0.0.1:8000` → مشخص می‌کند درخواست برای کدام هاست/پورت است.
- `User-Agent: curl/7.x` → معرفی کلاینت.
- `Accept: application/json` → کلاینت JSON می‌خواهد.

**Response Headers** نمونه:
- `Content-Type: application/json` → فرمت بدنهٔ پاسخ JSON است.
- `Content-Length: 36` → اندازهٔ بدنه (برای مدیریت شبکه مهم است).
- `Server: uvicorn` → نام نرم‌افزار سرور.

**چرا مهم‌اند؟**
- بدون هدرها، طرفین نمی‌دانند دادهٔ ردوبدل‌شده چه فرمتی دارد و چگونه باید پردازش شود.
- برای امنیت، کش، CORS، احراز هویت و… هم حیاتی‌اند.

</div>



<div dir="rtl" align="right">

## ۵) جایگاه FastAPI، Uvicorn و `curl` در زنجیره

```
[کلاینت: curl/مرورگر]
   │
   ▼
HTTP Request  →  [سوکت شبکه/Kernel]  →  [Uvicorn: ASGI Server]
                                             │
                                             ▼
                                       [FastAPI app]
                                             │
                                             ▼
HTTP Response ←────────────────────────────────────
```

- **curl**: ابزار خط فرمان برای ارسال درخواست (Client).
- **Uvicorn**: وب‌سرور ASGI که درخواست‌ها را از سوکت شبکه می‌گیرد و به اپ ASGI می‌دهد.
- **FastAPI**: لایهٔ اپلیکیشن (Routing/Validation/Serialization/Response).

</div>



<div dir="rtl" align="right">

## 🧪 تمرین‌ها

1. در کد بالا، مسیر `/people` را طوری تغییر بده که یک Query String ساده بگیرد:
   - ورودی: `min_age: int | None = None`
   - خروجی: اگر `min_age` داده شد، فقط افراد بالای آن سن را برگردان.

2. با `curl` هر دو حالت را تست کن:
```
curl.exe -i "http://127.0.0.1:8000/people"
curl.exe -i "http://127.0.0.1:8000/people?min_age=25"
```

3. هدر دلخواه به پاسخ اضافه کن (مثلاً `X-App: Demo`):
   - راهنمایی: از `from fastapi import Response` استفاده کن و قبل از برگرداندن، هدر را روی `response.headers["X-App"] = "Demo"` ست کن.

4. یک پاسخ خطای ساختگی بساز:
   - اگر `min_age` منفی بود، `HTTP 400` با بدنهٔ JSON برگردان (راهنمایی: `from fastapi import HTTPException`).

</div>


In [ ]:
# نسخه‌ی پیشرفته‌تر با Query Params، هدر سفارشی و خطای کنترل‌شده
from typing import Optional

from fastapi import FastAPI, HTTPException, Response

app = FastAPI()

PEOPLE = [
    {"name": "Ali", "age": 20},
    {"name": "Sara", "age": 27},
    {"name": "Reza", "age": 33},
]


@app.get("/people")
def read_people_2(min_age: Optional[int] = None, response: Response = None):
    if min_age is not None and min_age < 0:
        raise HTTPException(status_code=400, detail="min_age must be non-negative")
    result = PEOPLE
    if min_age is not None:
        result = [p for p in PEOPLE if p["age"] >= min_age]
    if response is not None:
        response.headers["X-App"] = "Demo"
    return {"people": result}


<div dir="rtl" align="right">

## ❓ سؤالات تشریحی + پاسخ‌های تو

**سؤال ۱:** اجزای اصلی یک HTTP Request چیست؟  
**پاسخ تو:** Request Line (method, path, protocol version)؛ Headers (Host, User-Agent, Accept)؛ Body (معمولاً در POST/PUT). ✅

**سؤال ۲:** تفاوت GET و POST در بدنهٔ درخواست؟  
**پاسخ تو:** GET عملاً Body ندارد؛ POST داده را در Body می‌فرستد. ✅ (نکته: از نظر استاندارد GET *می‌تواند* Body داشته باشد، ولی در عمل نادیده گرفته می‌شود.)

**سؤال ۳:** چرا FastAPI خروجی را خودکار به JSON تبدیل می‌کند؟  
**پاسخ تو:** برای تفسیر درست توسط کلاینت‌ها و استاندارد رایج APIها (JSON). ✅

**سؤال ۴:** اهمیت `Content-Type: application/json` در پاسخ؟  
**پاسخ تو:** به کلاینت می‌گوید بدنه JSON است و چگونه تفسیر شود. ✅

</div>



<div dir="rtl" align="right">

## ▶️ اسلات بعدی (A-2)
- **I/O Models**، **Sockets** و ساخت یک echo-server ساده (نسخهٔ blocking و نسخهٔ async با `asyncio.start_server`).
- هدف: لمس تفاوت blocking/non-blocking و آماده‌شدن برای درک event loop و کار Uvicorn.

</div>
